<a href="https://colab.research.google.com/github/ysjgithub/dl/blob/master/vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
from torchsummary import summary
import torch.optim as optim
import numpy as np
from torchvision import datasets, transforms,models



def panel3X3(input_channel,out_channel):
    return nn.Sequential(
        nn.Conv2d(input_channel, out_channel,3, stride=1,padding=1, bias=True),
        nn.BatchNorm2d(out_channel),
        nn.ReLU(inplace=True),
    )

class Vgg11(nn.Module):
    def __init__(self):
        super(Vgg11,self).__init__()
        self.s = nn.Sequential(
            panel3X3(3,64),

            nn.MaxPool2d(2, 2),

            panel3X3(64,128),

            nn.MaxPool2d(2, 2),

            panel3X3(128,256),
            panel3X3(256,256),

            nn.MaxPool2d(2, 2),

            panel3X3(256, 512),
            panel3X3(512, 512),

            nn.MaxPool2d(2,2),

            panel3X3(512, 512),
            panel3X3(512, 512),

            nn.MaxPool2d(2,2),

            nn.Flatten(),
            nn.Linear(512,512),
            nn.Linear(512,512),
            nn.Linear(512,10),
        )

    def forward(self, x):
        out  = self.s(x)
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net = Vgg11().to(device)
summary(net, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5          [-1, 128, 16, 16]          73,856
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
         MaxPool2d-8            [-1, 128, 8, 8]               0
            Conv2d-9            [-1, 256, 8, 8]         295,168
      BatchNorm2d-10            [-1, 256, 8, 8]             512
             ReLU-11            [-1, 256, 8, 8]               0
           Conv2d-12            [-1, 256, 8, 8]         590,080
      BatchNorm2d-13            [-1, 256, 8, 8]             512
             ReLU-14            [-1, 25

In [0]:

# Training dataset
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='.', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.RandomCrop(32,padding=4),
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), #R,G,B每层的归一化用到的均值和方差
                   ])), batch_size=128, shuffle=True, num_workers=4)
# Test dataset
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='.', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), #R,G,B每层的归一化用到的均值和方差
    ])), batch_size=128, shuffle=True, num_workers=4)




Extracting ./cifar-10-python.tar.gz to .


In [0]:
# optimizer = optim.Adam(model.parameters(), weight_decay=0.001)
# model = models.resnet18(num_classes=10).to(device)
# model =net
optimizer  = optim.SGD(model.parameters(),lr = 0.001,weight_decay =0.001)
criterion = nn.CrossEntropyLoss().to(device)
summary(model,(3,32,32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5          [-1, 128, 16, 16]          73,856
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
         MaxPool2d-8            [-1, 128, 8, 8]               0
            Conv2d-9            [-1, 256, 8, 8]         295,168
      BatchNorm2d-10            [-1, 256, 8, 8]             512
             ReLU-11            [-1, 256, 8, 8]               0
           Conv2d-12            [-1, 256, 8, 8]         590,080
      BatchNorm2d-13            [-1, 256, 8, 8]             512
             ReLU-14            [-1, 25

In [0]:

def one_hot(x, K):
    return np.array(x[:, None] == np.arange(K)[None, :], dtype=int)

def accuracy(data_loader):
    total_correct = 0
    for x, y in data_loader:
      x = x.to(device)
      y = one_hot(np.array(y.numpy()), 10)
      target_class = np.argmax(y, axis=1)
      predicted_class = np.argmax(model(x).cpu().detach().numpy(), axis=1)
      total_correct += np.sum(predicted_class == target_class)
    return total_correct / len(data_loader.dataset)
# accuracy(train_loader)


In [0]:
import time

def train(epoch):
    t = time.time()
    for x, y in train_loader:
      optimizer.zero_grad()
      x, y = x.to(device), y.to(device)
      output = model(x)
      loss = criterion(output, y)
      loss.backward()
      optimizer.step()
    t = time.time()-t
    train_accuracy_rate = accuracy(train_loader)
    test_accuracy_rate = accuracy(test_loader)
    print('epoch:{:04d}|loss:{:.4f}|'
      'train_accuracy:{:.4f}|test_accuracy:{:.4f}|time:{}'.format(epoch,loss,train_accuracy_rate,test_accuracy_rate,t))
#   
# A simple test procedure to measure STN the performances on MNIST.
#
# train(1)

In [0]:
for i in range(80):
  train(i)

epoch:0000|loss:0.0015|train_accuracy:0.9999|test_accuracy:0.9046|time:13.150197505950928
epoch:0001|loss:0.0032|train_accuracy:0.9998|test_accuracy:0.9052|time:13.09003472328186
epoch:0002|loss:0.0027|train_accuracy:0.9999|test_accuracy:0.9056|time:12.979182720184326
epoch:0003|loss:0.0087|train_accuracy:0.9999|test_accuracy:0.9070|time:12.912832736968994
epoch:0004|loss:0.0169|train_accuracy:0.9997|test_accuracy:0.9068|time:13.157330751419067
epoch:0005|loss:0.0029|train_accuracy:0.9999|test_accuracy:0.9053|time:12.841202974319458
epoch:0006|loss:0.0061|train_accuracy:0.9998|test_accuracy:0.9081|time:12.798437595367432
epoch:0007|loss:0.0026|train_accuracy:0.9999|test_accuracy:0.9065|time:12.69880723953247
epoch:0008|loss:0.0025|train_accuracy:0.9999|test_accuracy:0.9061|time:12.859669923782349
epoch:0009|loss:0.0013|train_accuracy:0.9999|test_accuracy:0.9093|time:12.697729587554932
epoch:0010|loss:0.0023|train_accuracy:0.9999|test_accuracy:0.9073|time:12.785932302474976
epoch:0011|l

In [0]:
root = "./drive/My Drive/experiment/CIFRA10/vgg11.pth"

torch.save(model.state_dict(),root)
